In [1]:
import time

import cheetah
import numpy as np
import ocelot as oc

import ARESlatticeStage3v1_9 as aresstuff

math_op.py: module Numba is not installed. Install it if you want speed up correlation calculations


[INFO    ] : : beam.py: module NUMBA is not installed. Install it to speed up calculation
[INFO    ] : : high_order.py: module NUMBA is not installed. Install it to speed up calculation
[INFO    ] : : : : : : : : : : : radiation_py.py: module NUMBA is not installed. Install it to speed up calculation
[INFO    ] csr.py: module NUMBA is not installed. Install it to speed up calculation
[INFO    ] csr.py: module PYFFTW is not installed. Install it to speed up calculation.
[INFO    ] csr.py: module NUMEXPR is not installed. Install it to speed up calculation
[INFO    ] wake3D.py: module NUMBA is not installed. Install it to speed up calculation


initializing ocelot...
import: module NUMBA is not installed. Install it to speed up calculation
import: module PYFFTW is not installed. Install it to speed up calculation
import: module NUMEXPR is not installed. Install it to speed up calculation


In [3]:
binning = 4
screen_resolution = (int(2448 / binning), int(2040 / binning))
meter_per_pixel = (3.5488e-6 * binning, 2.5003e-6 * binning)
screen_bin_edges = (np.linspace(-screen_resolution[0]*meter_per_pixel[0]/2,
                                             screen_resolution[0]*meter_per_pixel[0]/2,
                                             screen_resolution[0]+1),
                                 np.linspace(-screen_resolution[1]*meter_per_pixel[1]/2,
                                             screen_resolution[1]*meter_per_pixel[1]/2,
                                             screen_resolution[1]+1))


cell = cheetah.utils.subcell_of_ocelot(aresstuff.cell, "AREASOLA1", "AREABSCR1")
aresstuff.areamqzm1.k1 = 5.5
aresstuff.areamqzm2.k1 = -5.5
aresstuff.areamqzm3.k1 = 0.0
# cell = [oc.Drift(l=3.0), oc.Quadrupole(l=1, k1=0), oc.Drift(l=0), oc.Drift(l=3)]
# cell = [oc.Drift(l=3.0)]

lattice = oc.MagneticLattice(cell)

In [4]:
particles = oc.generate_parray(nparticles=int(1e+5), sigma_x=175e-6, sigma_y=175e-6)

t1 = time.time()

navigator = oc.Navigator(lattice)
_, particles = oc.track(lattice, particles, navigator)
image = np.histogram2d(particles.x(), particles.y(),
                       bins=screen_bin_edges)[0].transpose()

t2 = time.time()

print(f"{t2 - t1} seconds")

z = 2.0460399999999996 / 2.0460399999999996 : applied:  0.2719879150390625 seconds


In [5]:
particles = oc.generate_parray(nparticles=int(1e+5), sigma_x=175e-6, sigma_y=175e-6)

t1 = time.time()

navigator = oc.Navigator(lattice)
navigator.unit_step = lattice.totalLen
_, particles = oc.track(lattice, particles, navigator)
image = np.histogram2d(particles.x(), particles.y(),
                       bins=screen_bin_edges)[0].transpose()

t2 = time.time()

print(f"{t2 - t1} seconds")

z = 2.0460399999999996 / 2.0460399999999996 : applied:  0.2255399227142334 seconds


In [6]:
segment = cheetah.Segment.from_ocelot(cell)
segment.AREABSCR1.is_active = True  # Turn screen on and off

particles = cheetah.ParticleBeam.from_parameters(n=int(1e+5), sigma_x=175e-6, sigma_y=175e-6)

t1 = time.time()

_ = segment(particles)
image = segment.AREABSCR1.reading

t2 = time.time()

print(f"{t2 - t1} seconds")

0.021596193313598633 seconds


In [7]:
segment

Segment([Drift(length=0.00, name="AREASOLA1"), Drift(length=0.18, name="Drift_AREASOLA1"), Quadrupole(length=0.12, k1=5.5, misalignment=(0, 0), name="AREAMQZM1"), Drift(length=0.43, name="Drift_AREAMQZM1"), Quadrupole(length=0.12, k1=-5.5, misalignment=(0, 0), name="AREAMQZM2"), Drift(length=0.20, name="Drift_AREAMQZM2"), VerticalCorrector(length=0.02, angle=0.0, name="AREAMCVM1"), Drift(length=0.20, name="Drift_AREAMCVM1"), Quadrupole(length=0.12, k1=0.0, misalignment=(0, 0), name="AREAMQZM3"), Drift(length=0.18, name="Drift_AREAMQZM3"), HorizontalCorrector(length=0.02, angle=0.0, name="AREAMCHM1"), Drift(length=0.45, name="Drift_AREAMCHM1"), Screen(resolution=(2448, 2040), pixel_size=(3.5488e-06, 2.5003e-06), binning=1, misalignment=(0, 0), name="AREABSCR1")])